# Linear Model

In [5]:
# Importiere Bibliotheken
import sys
import io
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [13]:
# Kodierte Daten einlesen
data_labeled = pd.read_pickle('data_modified/tweets_labeled.pkl')
data_labeled.head()

,User,Tweet,Retweet,RT_from,Orig_Tweet,Tweet_Participants,Reply,Text,Tags,Lang,Link,Label
Datetime,,,,,,,,,,,,
2018-01-18 17:07:00,@GeroldSchlegel,RT @redder66: @NatalieRickli @NoBillag Die Unt...,True,@redder66,False,"[@NatalieRickli, @NoBillag, @SVPch]",False,Die Unterstützung der für die #NoBillag ergibt...,"[NoBillag, NeinzuNoBillag]",de,[],1
2017-12-23 13:53:00,@derlamentierer,@ProBillag auch sie haben das konzept von soli...,False,NaN,False,[@ProBillag],True,auch sie haben das konzept von solidarität nic...,"[nobillag, jazunobillag, NeinzuNoBillag]",de,[https://t.co/iFLeHBuSFS],6
2017-12-05 19:47:00,@FlorianSchwab,RT @VitalSteiner: Was die Gegner von #NoBillag...,True,@VitalSteiner,False,[],False,Was die Gegner von #NoBillag mit solchen Bilde...,[NoBillag],de,[https://t.co/IKOSeftUlm],6
2018-01-17 07:08:00,@Schweizer2018,RT @realDanielVozar: Wenn #NoBillag abgelehnt ...,True,@realDanielVozar,False,[@LHeimgartner],False,"Wenn #NoBillag abgelehnt wird, wird sich gar n...","[NoBillag, JazuNoBillag]",de,[],5
2018-02-19 09:52:00,@PilgramsWords,RT @_macmike: #StarkesStück https://t.co/2L7uG...,True,@_macmike,False,"[@Blickch, @20min, @bazonline]",False,#StarkesStück #AfD #SVP #ARD #ZDF #ORF #SRF #...,"[StarkesStück, AfD, SVP, ARD, ZDF, ORF, SRF, s...",de,"[https://t.co/2L7uGmmVH0, https://t.co/5Wf6qJ1...",0


In [14]:
# Funktionen definieren um Stoppworte zu entfernen & Daten weiter zu bereinigen
nltk.download('stopwords')
stopword_set = set(stopwords.words("german"))
def preprocess(raw_text):
    res = raw_text
    res = res.replace('ä', 'ae')
    res = res.replace('ö', 'oe')
    res = res.replace('ü', 'ue')
    res = res.replace('Ä', 'Ae')
    res = res.replace('Ö', 'Oe')
    res = res.replace('Ü', 'Ue')
    res = res.replace('ß', 'ss')
    stopword_set = set(stopwords.words("german"))
    return " ".join([i for i in re.sub(r'[^a-zA-Z\s]', "", res).lower().split() if i not in stopword_set])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/scornehls/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Daten bereinigen und X und y Variablen legen
texts=[]
for txt in data_labeled['Text']:
    txt=preprocess(txt)
    texts.append(txt)
labels = data_labeled['Label'].values

In [17]:
# TF-IDF document-term matrix erstellen
from sklearn.feature_extraction.text import CountVectorizer

count_vect_x = CountVectorizer(min_df=1)
X_counts= count_vect_x.fit_transform(texts)
X_counts.shape

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)
X_tfidf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


<1034x5300 sparse matrix of type '<class 'numpy.float64'>'
	with 12311 stored elements in Compressed Sparse Row format>

In [18]:
# test and train Daten definieren
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.5, random_state=14)

In [19]:
# Logistic Regression ausführen
from sklearn import linear_model

model1 = linear_model.LogisticRegression()
model1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
# Genauigkeit der Vorhersage ermitteln
print("Accuracy: {}".format(model1.score(X_test, y_test)))

Accuracy: 0.38104448742746616
